In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv')

In [ ]:
df.head()

Encode categorical columns

In [ ]:
def preprocess_data(df_new):

    #Encode experience column
    df_new.loc[df_new['experience'] == '>20', 'experience'] = '21'
    df_new.loc[df_new['experience'] == '<1', 'experience'] = '0'
    df_new['experience'] = df_new['experience'].astype(float)

    #Encode education_level column
    # Using pandas factorize method for ordinal data
    categories = pd.Categorical(df_new['education_level'], categories=['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], ordered=True)
    # Order of labels set for data
    print(categories)
    # Factorizing the column data
    labels, unique = pd.factorize(categories, sort=True)
    df_new['education_level'] = labels

    #Encode last_new_job column
    categories = pd.Categorical(df_new['last_new_job'], categories=['never', '1', '2', '3', '4', '>4'], ordered=True)
    # Order of labels set for data
    print(categories)
    # Factorizing the column data
    labels, unique = pd.factorize(categories, sort=True)
    df_new['last_new_job'] = labels

    #Encode company_size column
    categories = pd.Categorical(df_new['company_size'], categories=['<10', '10/49', '50-99', '100-500', '500-999', '1000-4999', '5000-9999', '10000+'], ordered=True)
    # Order of labels set for data
    print(categories)
    # Factorizing the column data
    labels, unique = pd.factorize(categories, sort=True)
    df_new['company_size'] = labels
    
    #Fill the missing values in categorical columns
    df_new = df_new.fillna(df_new.mode().iloc[0])
    df_new.head()
    
    return df_new

In [ ]:
#Encode the experience column
df_new = preprocess_data(df.copy())
df_new.head()

In [ ]:
corr_mat = df_new.corr()
#print(corr_mat)
corr_mat

In [ ]:
#plot the correlation matrix heat map
sns.heatmap(corr_mat, annot=True)

# Feature - Target correlation based insights
1. Candidates working in Companies situated in the cities with higher development index (More developed cities) are less likely to switch their jobs.
2. Candidates with higher work experience are less likely to swith their jobs.
3. Candidates with higher work experience like to live in cities with higher higher development index.
4. Candidates with higher work experience has higher education level
5. Candidates working in large companies are less likely to switch their jobs

# Model Training with Weighted Logistic Regressi

In [ ]:
random_state = 0
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, recall_score

In [ ]:
# instantiate labelencoder object
le = LabelEncoder()

# Categorical boolean mask
categorical_feature_mask = df_new.dtypes==object
print(categorical_feature_mask)
# filter categorical columns using mask and turn it into a list
categorical_cols = df_new.columns[categorical_feature_mask].tolist()
print(categorical_cols)
# apply le on categorical feature columns
#df_new = df.copy()
df_new[categorical_cols] = df_new[categorical_cols].apply(lambda col: le.fit_transform(col))
df_new.head(10)

In [ ]:
df['target'] = df['target'].astype(int)
#check the class distribution
df['target'].value_counts()/df.shape[0]

> From the above cell we can see that target class disctribution is imbalanced.

In [ ]:
#get the dependent and independent variables
#df_new = df_new.drop(columns=['enrollee_id'])
df_new = df_new.drop(['enrollee_id'], axis=1)
df_features = df_new[df_new.columns[0:-1]]
df_target = df_new['target']
x_train, x_val, y_train, y_val = train_test_split(df_features, df_target, test_size=0.2, random_state=random_state)

In [ ]:
#Standardize the features
from sklearn import preprocessing
norm = preprocessing.StandardScaler()
df_features=norm.fit_transform(df_features)

In [ ]:
#Weighted logistic regression
w = {0:2, 1:7} #use the inverse of class ratios as class weights
lr_model_weighted = LogisticRegression(random_state=random_state, class_weight=w)
lr_model_weighted.fit(x_train, y_train)

In [ ]:
def test_model(model, x_test, y_test):
    predictions = model.predict(x_test)
    print("Predictions:")
    print(predictions)
    # performance
    print(f'Accuracy Score: {accuracy_score(y_test,predictions)}')
    print(f'Area Under Curve: {roc_auc_score(y_test, predictions)}')
    print(f'Recall score: {recall_score(y_test, predictions)}')
    print("Classification report: ")
    print(classification_report(y_test, predictions))
    cm = confusion_matrix(y_test, predictions)
    print("Confusion Matrix Plot: \n{}".format(cm))
    sns.heatmap(cm, annot=True)

In [ ]:
test_model(lr_model_weighted, x_val, y_val)

In [ ]:
#/kaggle/input/hr-analytics-job-change-of-data-scientists/sample_submission.csv
#/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_test.csv
df_test = pd.read_csv("/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_test.csv")
df_test.head()

In [ ]:
df_test_new = preprocess_data(df_test.copy())
df_test_new = df_test_new.drop(['enrollee_id'], axis=1)
# instantiate labelencoder object
le = LabelEncoder()

# Categorical boolean mask
categorical_feature_mask = df_test_new.dtypes==object
print(categorical_feature_mask)
# filter categorical columns using mask and turn it into a list
categorical_cols = df_test_new.columns[categorical_feature_mask].tolist()
print(categorical_cols)
# apply le on categorical feature columns
#df_new = df.copy()
df_test_new[categorical_cols] = df_test_new[categorical_cols].apply(lambda col: le.fit_transform(col))

#Standardize the features
from sklearn import preprocessing
norm = preprocessing.StandardScaler()
df_test_new=norm.fit_transform(df_test_new)

#df_test_new.head(10)

In [ ]:
pr = lr_model_weighted.predict(df_test_new)
print(pr)
df_test['target'] = pr
df_test.head()

In [ ]:
final_out = df_test[['enrollee_id','target']]
final_out.head()

In [ ]:
final_out.to_csv('./subm.csv',index=False)